In [1]:
import pandas as pd
import io
import re
import os

from stops_prn_extractor import StopsPRNExtractor

# Instantiate the extractor object
extractor = StopsPRNExtractor()


In [ ]:
# Instantiate the extractor object
extractor = StopsPRNExtractor()

prn_files_directory = "./stops_output_prn_report_files"

prn_files = [
    ("2024", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA_STOPSY2024Results.prn"),
    ("2045", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2045Results.prn"),
    ("2050", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2050Results.prn"),
    # Add more as needed
]

# Add files to the extractor
extractor.add_files(prn_files)

extractor.extract_table_9_01()
extractor.extract_table_10_01() 
extractor.extract_table_10_02() 

# Display results (optional, good for debugging)
for alias_key in extractor.tables:
    print(f"\nAlias: {alias_key}")
    print("Metadata:")
    for k, v in extractor.metadata[alias_key].items():
        print(f"  {k}: {v}")
    print(f"\nTable data for {alias_key}:")
    display(extractor.tables[alias_key])

# Export extracted tables to CSV files
extractor.export_to_csv(output_dir="./extracted_csv_tables")


Alias: 2024_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            10:48:37
  Program: STOPS

Table data for 2024_table_9_01:


,Stop_id1,Station_Name,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_XFR,Y2024_EXISTING_ALL,Y2024_NO-BUILD_WLK,Y2024_NO-BUILD_KNR,Y2024_NO-BUILD_PNR,Y2024_NO-BUILD_XFR,Y2024_NO-BUILD_ALL,Y2024_BUILD_WLK,Y2024_BUILD_KNR,Y2024_BUILD_PNR,Y2024_BUILD_XFR,Y2024_BUILD_ALL
0,A6456&T,Alewife,2485,460,202,824,3971,2485,460,202,824,3971,2485,460,202,824,3971
1,A6457&T,Davis,4046,229,0,512,4788,4046,229,0,512,4788,4046,229,0,512,4788
2,A6459&T,Porter,3832,213,0,1441,5485,3832,213,0,1441,5485,3832,213,0,1441,5485
3,A6461&T,Harvard,6203,172,0,1554,7929,6203,172,0,1554,7929,6203,172,0,1554,7929
4,A6463&T,Central,6570,214,0,711,7495,6570,214,0,711,7495,6570,214,0,711,7495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11327,A6639&T,Assembly,2163,30,0,0,2192,2163,30,0,0,2192,2163,30,0,0,2192
11328,A6428&T,Wellington,1921,367,1630,902,4819,1921,367,1630,902,4819,1921,367,1630,902,4819
11329,A6430&T,Malden Center,5680,487,5,2743,8916,5680,487,5,2743,8916,5680,487,5,2743,8916
11330,A6432&T,Oak Grove,2073,520,51,785,3429,2073,520,51,785,3429,2073,520,51,785,3429



Alias: 2045_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            16:58:08
  Program: STOPS

Table data for 2045_table_9_01:


,Stop_id1,Station_Name,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_XFR,Y2024_EXISTING_ALL,Y2045_NO-BUILD_WLK,Y2045_NO-BUILD_KNR,Y2045_NO-BUILD_PNR,Y2045_NO-BUILD_XFR,Y2045_NO-BUILD_ALL,Y2045_BUILD_WLK,Y2045_BUILD_KNR,Y2045_BUILD_PNR,Y2045_BUILD_XFR,Y2045_BUILD_ALL
0,A6456&T,Alewife,2485,460,202,824,3971,3195,484,162,901,4741,3195,484,162,901,4741
1,A6457&T,Davis,4046,229,0,512,4788,4176,220,0,769,5165,4176,220,0,769,5165
2,A6459&T,Porter,3832,213,0,1441,5485,3859,207,0,2969,7035,3859,207,0,2969,7035
3,A6461&T,Harvard,6203,172,0,1554,7929,6075,174,0,2108,8357,6075,174,0,2108,8357
4,A6463&T,Central,6570,214,0,711,7495,7243,231,0,829,8303,7243,231,0,829,8303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11327,A6639&T,Assembly,2163,30,0,0,2192,3272,26,0,217,3515,3272,26,0,217,3515
11328,A6428&T,Wellington,1921,367,1630,902,4819,2221,479,3077,1434,7211,2221,479,3077,1434,7211
11329,A6430&T,Malden Center,5680,487,5,2743,8916,6948,546,3,7400,14897,6948,546,3,7400,14897
11330,A6432&T,Oak Grove,2073,520,51,785,3429,1916,560,8,256,2740,1916,560,8,256,2740



Alias: 2050_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for 2050_table_9_01:


...


Alias: 2045
Metadata:
  Run: Boston_Regional_STOPS                                                                                            16:58:08
  Program: STOPS

Table data for 2045:


,Route_ID,Route_Name,Count,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_ALL,Y2045_NO-BUILD_WLK,Y2045_NO-BUILD_KNR,Y2045_NO-BUILD_PNR,Y2045_NO-BUILD_ALL,Y2045_BUILD_WLK,Y2045_BUILD_KNR,Y2045_BUILD_PNR,Y2045_BUILD_ALL
0,04N&M,--RT04N-RT04N-Waverly St Natic,0,299,6,2,307,307,4,1,312,307,4,1,312
1,04S&M,--RT04S-RT04S-Central Hub Beav,0,147,2,1,150,155,1,1,157,155,1,1,157
2,07C&M,--RT07C-RT07C,0,255,26,4,285,273,27,4,305,273,27,4,305
3,1&M,--RT01-RT01-Woodland Green Lin,0,144,4,2,150,155,3,2,161,155,3,2,161
4,1&T,--1-Harvard Square - Nubian St,9490,9361,40,172,9573,11195,48,97,11340,11195,48,97,11340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,Mattapan&T,--Mattapan Trolley-Mattapan Tr,4523,3416,289,896,4601,4871,341,1660,6872,4871,341,1660,6872
251,NCS&M,--Natick Commuter Shuttle-Nati,0,8,0,1,10,8,0,1,10,8,0,1,10
252,Orange&T,--Orange Line-Orange Line-Rapi,115769,93982,8294,12794,115071,117088,10381,26508,153977,117088,10381,26508,153977
253,Red&T,--Red Line-Red Line-Rapid Tran,133995,109776,8110,12642,130528,126546,9829,25890,162266,126546,9829,25890,162266



Alias: 2050
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for 2050:


,Route_ID,Route_Name,Count,Y2024_EXISTING_WLK,Y2024_EXISTING_KNR,Y2024_EXISTING_PNR,Y2024_EXISTING_ALL,Y2050_NO-BUILD_WLK,Y2050_NO-BUILD_KNR,Y2050_NO-BUILD_PNR,Y2050_NO-BUILD_ALL,Y2050_BUILD_WLK,Y2050_BUILD_KNR,Y2050_BUILD_PNR,Y2050_BUILD_ALL
0,04N&M,--RT04N-RT04N-Waverly St Natic,0,299,6,2,307,313,4,1,319,313,4,1,319
1,04S&M,--RT04S-RT04S-Central Hub Beav,0,147,2,1,150,155,1,1,158,155,1,1,158
2,07C&M,--RT07C-RT07C,0,255,26,4,285,268,26,4,298,268,26,4,298
3,1&M,--RT01-RT01-Woodland Green Lin,0,144,4,2,150,151,3,2,156,151,3,2,156
4,1&T,--1-Harvard Square - Nubian St,9490,9361,40,172,9573,11416,48,95,11559,11416,48,95,11559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,Mattapan&T,--Mattapan Trolley-Mattapan Tr,4523,3416,289,896,4601,4884,349,1654,6887,4884,349,1654,6887
251,NCS&M,--Natick Commuter Shuttle-Nati,0,8,0,1,10,8,0,1,10,8,0,1,10
252,Orange&T,--Orange Line-Orange Line-Rapi,115769,93982,8294,12794,115071,117748,10476,26393,154617,117748,10476,26393,154617
253,Red&T,--Red Line-Red Line-Rapid Tran,133995,109776,8110,12642,130528,128384,9948,25781,164113,128384,9948,25781,164113


Exported '2024_table_9_01' to './extracted_csv_tables\2024_Table_9_01.csv' successfully.
Exported '2045_table_9_01' to './extracted_csv_tables\2045_Table_9_01.csv' successfully.
Exported '2050_table_9_01' to './extracted_csv_tables\2050_Table_9_01.csv' successfully.
Exported '2024' to './extracted_csv_tables\2024_Table_10_01.csv' successfully.
Exported '2045' to './extracted_csv_tables\2045_Table_10_01.csv' successfully.
Exported '2050' to './extracted_csv_tables\2050_Table_10_01.csv' successfully.
